# Paper Implemention

In [1]:
# Step 1: Data Preprocessing
# Load and preprocess Brain Stroke dataset

# Step 2: Feature Extraction
# Use pre-trained CNN models (vgg16) to extract features from images

# Step 3: Genetic Algorithm
# Implement genetic algorithm for feature selection

# Step 4: BiLSTM Model
# Implement BiLSTM model for stroke prediction

# Step 5: Evaluation
# Evaluate model performance using metrics like accuracy, precision, recall, etc.

# Step 6: Comparison
# Compare your model's performance with other approaches

# Additional steps: Visualization, Hyperparameter tuning, Model optimization, etc.

In [2]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from sklearn.model_selection import train_test_split

## Step 1: Data Preprocessing

In [3]:
normal_path = "Brain_Data_Organised/Normal"
stroke_path = "Brain_Data_Organised/Stroke"
normal_folder = os.listdir(normal_path)
stroke_folder = os.listdir(stroke_path)
print("Images in Normal Data:", len(normal_folder))
print("Images in Stroke Data:", len(stroke_folder))

Images in Normal Data: 1551
Images in Stroke Data: 950


## Step 2: Feature Extraction

In [4]:
def load_and_preprocess_image(image_path):
    # Load and preprocess image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

In [5]:

# Load VGG16 model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features
def extract_features(image):
    features = base_model.predict(np.expand_dims(image, axis=0))
    return features

# Load and preprocess images
image_paths = [os.path.join(normal_path, img) for img in normal_folder] + \
              [os.path.join(stroke_path, img) for img in stroke_folder]

X_features = []
for img_path in image_paths:
    img = load_and_preprocess_image(img_path)
    features = extract_features(img)
    X_features.append(features)

X_features = np.array(X_features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━

## Step 3: Genetic Algorithm for Feature Selection

In [6]:
# Initialize parameters for GA
population_size = 50
num_generations = 10
mutation_rate = 0.1

# Fitness evaluation function (example: accuracy of a model)
def fitness(chromosome):
    # Assuming you have a function to evaluate the fitness of a chromosome
    return np.sum(chromosome)

# Genetic Algorithm functions
def initialize_population(population_size, chromosome_length):
    return np.random.randint(2, size=(population_size, chromosome_length))

def selection(population, scores):
    sorted_indices = np.argsort(scores)[::-1]  # Sort in descending order
    selected_indices = sorted_indices[:int(0.2 * len(sorted_indices))]  # Select top 20%
    return population[selected_indices]

def crossover(parents):
    offspring = []
    for _ in range(population_size - len(parents)):
        parent1, parent2 = random.choices(parents, k=2)
        if len(parent1) == 1 and len(parent2) == 1:
            crossover_point = 0
        else:
            crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
        offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        offspring.append(offspring1)
        offspring.append(offspring2)
    return np.array(offspring)



def mutation(population):
    for i in range(len(population)):
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(population[i]) - 1)
            population[i][mutation_point] = 1 - population[i][mutation_point]  # Flip the bit
    return population

# Initialize population
population = initialize_population(population_size, X_features.shape[1])

# Genetic Algorithm main loop
for generation in range(num_generations):
    # Evaluate fitness
    scores = [fitness(chromosome) for chromosome in population]
    
    # Select parents
    parents = selection(population, scores)
    
    # Crossover
    offspring = crossover(parents)
    
    # Mutation
    offspring = mutation(offspring)
    
    # Replace population with offspring
    population = offspring

## Step 4: BiLSTM Model for Stroke Prediction

In [7]:
# Load labels
y = np.concatenate([np.zeros(len(normal_folder)), np.ones(len(stroke_folder))])

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [8]:
print("Before reshaping - X_train shape:", X_train.shape)
print("Before reshaping - X_test shape:", X_test.shape)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

print("After reshaping - X_train shape:", X_train.shape)
print("After reshaping - X_test shape:", X_test.shape)


Before reshaping - X_train shape: (2000, 1, 7, 7, 512)
Before reshaping - X_test shape: (501, 1, 7, 7, 512)
After reshaping - X_train shape: (2000, 1, 25088)
After reshaping - X_test shape: (501, 1, 25088)


In [9]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=X_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

d:\Coding\A Machine Learning-Based Diagnostic Model Using Neuroimages\myenv\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.5801 - loss: 0.6798 - val_accuracy: 0.6088 - val_loss: 0.6687
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.6417 - loss: 0.6563 - val_accuracy: 0.6088 - val_loss: 0.6631
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.6180 - loss: 0.6615 - val_accuracy: 0.6088 - val_loss: 0.6698
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6272 - loss: 0.6611 - val_accuracy: 0.6088 - val_loss: 0.6627
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6312 - loss: 0.6536 - val_accuracy: 0.6088 - val_loss: 0.6680
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6125 - loss: 0.6738 - val_accuracy: 0.6088 - val_loss: 0.6626
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6244 - loss: 0.6568 - val_accuracy: 0.6088 - val_loss: 0.6625
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.6323 - loss: 0.6552 - val_accuracy: 0.6088 - v

## Step 5: Evaluation

In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6212 - loss: 0.6532
Test Accuracy: 0.6127744317054749


In [ ]:
# Step 6: Comparison
# Compare your model's performance with other approaches..........................................................